# Data Exploration
**Purpose**: Explore the financial_data.db database and understand available data

**Date**: 2026-02-22

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set up paths
project_root = Path.cwd().parent
db_path = project_root / 'data' / 'financial_data.db'

print(f"Project root: {project_root}")
print(f"Database: {db_path}")
print(f"Database exists: {db_path.exists()}")

## 1. Database Schema Overview

In [ ]:
# Connect to database
conn = sqlite3.connect(db_path)

# Get all tables
tables_query = "SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;"
tables = pd.read_sql_query(tables_query, conn)

print(f"Found {len(tables)} tables:")
print(tables)

## 2. Stock Price Data

In [ ]:
# Get stock price table schema
schema_query = "PRAGMA table_info(fact_stock_price);"
schema = pd.read_sql_query(schema_query, conn)
print("Stock Price Table Schema:")
print(schema[['name', 'type', 'notnull']])

In [ ]:
# Count total stock prices
count_query = "SELECT COUNT(*) as count FROM fact_stock_price;"
count = pd.read_sql_query(count_query, conn)
print(f"Total stock price records: {count['count'].iloc[0]:,}")

In [ ]:
# Get date range
date_range_query = """
SELECT 
    MIN(d.date) as min_date,
    MAX(d.date) as max_date,
    COUNT(DISTINCT d.date) as n_dates
FROM fact_stock_price s
JOIN dim_date d ON s.date_id = d.date_id;
"""
date_range = pd.read_sql_query(date_range_query, conn)
print("\nDate Range:")
print(date_range)

In [ ]:
# Get available tickers
tickers_query = """
SELECT DISTINCT c.ticker, c.company_name
FROM fact_stock_price s
JOIN dim_company c ON s.company_id = c.company_id
ORDER BY c.ticker;
"""
tickers = pd.read_sql_query(tickers_query, conn)
print(f"\nTotal tickers available: {len(tickers)}")
print("\nFirst 20 tickers:")
print(tickers.head(20))

## 3. Check Sentinel Sector Tickers

In [ ]:
# Define sector tickers
sector_tickers = {
    'tech': ['AAPL', 'MSFT', 'NVDA', 'GOOGL', 'META', 'TSLA', 'AMD', 'INTC', 'CRM', 'ADBE'],
    'finance': ['JPM', 'BAC', 'GS', 'MS', 'WFC', 'C', 'BLK', 'AXP', 'USB', 'PNC'],
    'crypto': ['BTC-USD', 'ETH-USD', 'SOL-USD', 'BNB-USD', 'ADA-USD'],
    'commodities': ['GLD', 'SLV', 'USO', 'DBC'],
    'cyclicals': ['CAT', 'DE', 'BA', 'HON', 'MMM', 'GE', 'UPS', 'FDX', 'DAL', 'UAL']
}

# Check which tickers are available
available_tickers = set(tickers['ticker'].tolist())

for sector, ticker_list in sector_tickers.items():
    available = [t for t in ticker_list if t in available_tickers]
    missing = [t for t in ticker_list if t not in available_tickers]
    
    print(f"\n{sector.upper()}:")
    print(f"  Available: {len(available)}/{len(ticker_list)}")
    if available:
        print(f"  Tickers: {', '.join(available)}")
    if missing:
        print(f"  Missing: {', '.join(missing)}")

## 4. Sample Data for Training Period (2019-2024)

In [ ]:
# Load sample data for tech sector
sample_query = """
SELECT 
    d.date,
    c.ticker,
    s.open_price as open,
    s.high_price as high,
    s.low_price as low,
    s.close_price as close,
    s.adjusted_close,
    s.volume,
    s.price_change_percent
FROM fact_stock_price s
JOIN dim_date d ON s.date_id = d.date_id
JOIN dim_company c ON s.company_id = c.company_id
WHERE 
    c.ticker IN ('AAPL', 'MSFT', 'GOOGL')
    AND d.date BETWEEN '2019-01-01' AND '2024-12-31'
ORDER BY d.date, c.ticker;
"""

sample_df = pd.read_sql_query(sample_query, conn)
sample_df['date'] = pd.to_datetime(sample_df['date'])

print(f"Sample data loaded: {len(sample_df)} rows")
print(f"Date range: {sample_df['date'].min()} to {sample_df['date'].max()}")
print(f"\nSample:")
print(sample_df.head(10))

In [ ]:
# Check data completeness per ticker
completeness = sample_df.groupby('ticker').agg({
    'date': ['count', 'min', 'max'],
    'close': lambda x: x.isna().sum()
})
completeness.columns = ['n_records', 'first_date', 'last_date', 'missing_close']
print("\nData Completeness:")
print(completeness)

## 5. Visualize Sample Data

In [ ]:
# Plot adjusted close prices
plt.figure(figsize=(14, 6))

for ticker in sample_df['ticker'].unique():
    ticker_df = sample_df[sample_df['ticker'] == ticker]
    plt.plot(ticker_df['date'], ticker_df['adjusted_close'], label=ticker, alpha=0.7)

plt.title('Adjusted Close Prices (2019-2024)', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate and plot daily returns
sample_df['returns'] = sample_df.groupby('ticker')['adjusted_close'].pct_change()

plt.figure(figsize=(14, 6))

for ticker in sample_df['ticker'].unique():
    ticker_df = sample_df[sample_df['ticker'] == ticker]
    plt.plot(ticker_df['date'], ticker_df['returns'], label=ticker, alpha=0.5)

plt.title('Daily Returns (2019-2024)', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Daily Return')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.show()

## 6. Summary Statistics

In [ ]:
# Calculate summary statistics per ticker
stats = sample_df.groupby('ticker')['returns'].agg([
    ('mean', 'mean'),
    ('std', 'std'),
    ('min', 'min'),
    ('max', 'max'),
    ('sharpe', lambda x: (x.mean() / x.std()) * np.sqrt(252))  # Annualized Sharpe
])

print("\nSummary Statistics (Daily Returns):")
print(stats)
print("\nNote: Returns are daily, Sharpe ratio is annualized")

## 7. Close Connection

In [ ]:
conn.close()
print("Database connection closed")

## Next Steps

1. ✅ Database is accessible and contains data
2. ⏳ Create data loader module (`src/data/data_loader.py`)
3. ⏳ Test data loader with sectors
4. ⏳ Begin model training

See `02_data_loader_implementation.ipynb` to build the data loader.